## Постановка задачи
Требуется предсказать ожидаемую продолжительность жизни на 2019 год в РФ, пердварительно выявив факторы, в наибольшей степени влияющие на этот показатель.

## Описание
Соберите случайный ансамбль из нескольких методов выделения факторов - корреляции, взаимной информации, важности признаков, главных компонент, независимых компонент и др. Получите не менее 3 наборов из 5 наиболее важных признаков.

Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.

Используя эти ансамбли, предскажите продолжительность жизни на 2019 год.

Данные:
video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv

### Подключение библиотек

In [ ]:
from sklearn.decomposition import PCA, FastICA
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
%matplotlib inline


In [ ]:
def linear_extrapolation(x):
    X = np.array(x.dropna().index.astype(int)).reshape(-1, 1)
    Y = np.array(x.dropna().values).reshape(-1, 1)
    if X.shape[0] > 0:
        f = LinearRegression().fit(X, Y)
        for i in x.index:
            v = x.loc[i]
            if v != v:
                v = f.predict([[int(i)]])[0][0]
                if v < 0:
                    v = 0
                x.loc[i] = v
    return x

def make_ensemble(x, y):
    pca10 = PCA(n_components=10, random_state=42).fit(x)
    pca10_feature_importances = np.zeros(len(pca10.components_[0]))
    for c in range(len(pca10.components_)):
        pca10_feature_importances += pca10.components_[c]

    ica10 = FastICA(n_components=8, random_state=42).fit(x)
    ica10_feature_importances = np.zeros(len(ica10.components_[0]))
    for c in range(len(ica10.components_)):
        ica10_feature_importances += ica10.components_[c]

    return {
        'rfr250': np.argsort(RandomForestRegressor(n_estimators=250, random_state=17).fit(x, y).feature_importances_)[::-1],
        'etr250': np.argsort(ExtraTreesRegressor(n_estimators=250, random_state=17).fit(x, y).feature_importances_)[::-1],
        'pca10': np.argsort(np.abs(pca10_feature_importances))[::-1],
        'ica10': np.argsort(np.abs(ica10_feature_importances))[::-1]
    }


### Загрузка и очистка данных
Линейно интерполируем пропуски, а начальные и конечные пропущенные данные экстраполируем по линейному закону. Удаляем пустые параметры

In [ ]:
data = pd.read_csv("./data/rosstat.csv", na_values=["-", " - ", "...", "…", " -"])

In [ ]:
# data[data['feature']=='ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)']['2019']

In [ ]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)
data = data.apply(linear_extrapolation, axis=1, result_type="expand")

In [ ]:
data["feature"] = features
data.dropna(inplace=True)
features = np.array(data["feature"])
data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)

In [ ]:
# Нормируем данные для сопоставления влияния разных факторов
scaller = MinMaxScaler()
data = pd.DataFrame(scaller.fit_transform(data))
data.columns = features
data.head(10)

### Случайные наборы признаков

In [ ]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
             "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

Обучим несколько моделей на случайном поднаборе признаков, отранжируем признаки по значимости, возьмем 10 наиболее значащих признаков, повторим такой выбор несколько раз, затем выберем топ10 из нескольких итераций

In [ ]:
super_features_n = 3
super_features = []
subset_size = 20  # 50
subset_power = 5  # 100
subset_iter = 30  # 50
subset_super = 10  # 20

for n in range(super_features_n):
    super_features.append(np.zeros(len(features)))
    for k in range(subset_super):
        subset_features = np.zeros(len(features))
        for j in range(subset_iter):
            features_ensemble = [0]*len(features)
            for f_random in np.random.randint(len(features), size=[subset_power, subset_size]):
                x_subset = data[features[f_random]].copy()
                if y_column in features[f_random]:
                    x_subset = x_subset.drop(labels=[y_column], axis=1)
                for y_column_ in y_columns:
                    if y_column_ in features[f_random]:
                        x_subset = x_subset.drop(labels=[y_column_], axis=1)
                ensemble = make_ensemble(x_subset, y)
                for e in ensemble:
                    i = 0
                    for f in ensemble[e]:
                        features_ensemble[f_random[f]] += subset_size - i
                        i += 1
            subset_features[np.argsort(features_ensemble)[::-1][:10]] += 1
        super_features[n][np.argsort(subset_features)[::-1][:10]] += 1


In [ ]:
for xsf in super_features:
    print(np.argsort(xsf)[::-1][:5])


In [ ]:
for xsf in super_features:
    print(features[np.argsort(xsf)[::-1][:5]])
    print()

## Предсказание на 2019
**По итогам 2019 года средняя продолжительность жизни в России достигла очередного исторического максимума и составила 73,4 года**

In [ ]:
y = data[y_column]

for features_ in super_features:
    main_params = np.argsort(features_)[::-1][:5]
    x = data[[features[i] for i in main_params]]

    # Сдвинем факторы на год назад: чтобы предсказывать по значениям текущего года - следующий
    x_ = x[:-1]
    y_ = np.array(y[1:])

    # Строим ансамбли
    ensemble_lr = []
    for i in range(100):
        ensemble_lr.append(LinearRegression().fit(x_, y_))

    ensemble_et = []
    for i in range(100):
        ensemble_et.append(ExtraTreesRegressor(
            n_estimators=50, random_state=i).fit(x_, y_))

    ensemble_cb = []
    for i in range(100):
        ensemble_cb.append(CatBoostRegressor(
            iterations=20, depth=16, random_state=i, verbose=False).fit(x_, y_))

    # Вычисляем итоговое предсказание
    prediction = 0
    for i in range (100):
        prediction += ensemble_lr[i].predict(x[-1:])
        prediction += ensemble_et[i].predict(x[-1:])
        prediction += ensemble_cb[i].predict(x[-1:])
    prediction /= 300

    # print (prediction)
    # Маштабируем предсказание
    data_predict = data.copy()
    data_predict[y_column] = np.ones(len(data_predict)) * prediction
    data_predict = scaller.inverse_transform(data_predict)
    
    prediction = data_predict[0][np.where(features==y_column)][0]
    print(main_params, round(prediction, 4))